# Clean USIS Dataset
**Author:** Jacob Kvasnicka <br>
**Date:** September 24, 2024

## Setup

In [8]:
import pandas as pd
import numpy as np
import random
import json

from config_management import UnifiedConfiguration
from raw_processing.cehd_cleaning import CehdCleaner
from raw_processing import cehd_loading
from raw_processing import usis
from raw_processing.tests import test_cehd, test_usis

config = UnifiedConfiguration()

In [9]:
# Clean dupliates in USIS
# Determine which years to focus on
# Extract NAICS and see which sectors/subsectors covered
# Prepare target
# Build ML models for CEHD, USIS, and the union

In [17]:
expected_cehd = test_cehd.load_test_data(config.path)

In [11]:
raw_cehd = cehd_loading.raw_chem_exposure_health_data(config.cehd, config.path)

cehd_cleaner = CehdCleaner(config.path, config.cehd)

cehd_data = cehd_cleaner.clean_raw_data(raw_cehd)

In [18]:
expected_usis = test_usis.load_test_data(config.path)

In [12]:
raw_usis = usis.load_raw_usis_data(config.path['raw_usis_file'])

usis_cleaner = usis.UsisCleaner(config.path, config.usis)

usis_data = usis_cleaner.clean_raw_data(raw_usis)